### Notebook - Running Spark on OCI Dataflow using PySpark and Spark SQL
<details>
<summary><font size="2">Check OCI Dataflow documentation for policies required;</font></summary>
<li><a href="https://docs.public.oneportal.content.oci.oraclecloud.com/en-us/iaas/data-flow/using/policies-data-flow-studio.htm#policies-data-flow-studio">Policies Required to Integrate Data Flow and Data Science</a></li>
</details><details><summary><font size="2">Check that the data science notebook session has privileges on OCI Dataflow family and any dependencies;</font></summary>

```ALL {resource.type = 'datasciencenotebooksession', resource.compartment.id = '<compartment_id>'}```

</details><details><summary><font size="2">Policy for Data Science to manage OCI Data Flow</font></summary>

```allow dynamic-group <ds-dynamic-group> to manage dataflow-family in compartment '<your-compartment-name>'```

</details><details><summary><font size="2">Policy statements for logging</font></summary>

```allow dynamic-group <ds-dynamic-group> to manage log-groups in compartment '<your-compartment-name>'```
```allow dynamic-group <ds-dynamic-group> to manage log-content in compartment '<your-compartment-name>'```
</details>

In [6]:
import ads
ads.set_auth("resource_principal") # Supported values: resource_principal, api_key

In [7]:
# Replace with your valuues;
#  compartment id for creating the compute/application in OCI Dataflow
#  bucket for writing OCI Dataflow logs
#  namespace your OCI Object Storage namespace
#  driver_shape ie. VM.Standard.E4.Flex
#  executor_shape ie. VM.Standard.E4.Flex
# 
#
#Non- autoscaling create session command in New Compute - this will print a session id that you could reconnect to  
import json
command = {
    "compartmentId": "<your_compartment_id>",
    "displayName": "Data Analysis in Python",
    "language": "PYTHON",
    "sparkVersion": "3.2.1",
    "driverShape": "<driver_shape>",
    "executorShape": "<executor_shape>",
    "driverShapeConfig":{"ocpus":1,"memoryInGBs":16},
    "executorShapeConfig":{"ocpus":1,"memoryInGBs":16},
    "numExecutors": 1,
    "type": "SESSION",
    "logsBucketUri": "oci://<bucket>@<namespace>/",
    "configuration": {"spark.oracle.datasource.enabled":"true"}
}
command = f'\'{json.dumps(command)}\''

In [8]:
load_ext dataflow.magics

The dataflow.magics extension is already loaded. To reload it, use:
  %reload_ext dataflow.magics


In [9]:
%help

Magic,Example,Explanation
create_session,"%create_session -l python -c '{""compartmentId"":""Data Flow Run resource compartment OCID"",""displayName"":""SessionApp"",""sparkVersion"":""3.2.1"",""driverShape"":""VM.Standard2.1"",""executorShape"":""VM.Standard2.1"",""numExecutors"":1,""archiveUri"":""Object Storage URL for Data Flow zip archive."",""metastoreId"":""optional metastore OCID"",""configuration"":{ ""spark.archives"":""oci://bucket@namespace/path/to/conda/pack"", #optional property to use Dataflow 'Run' resource to access OCI resources. ""dataflow.auth"":""resource_principal"" }}'","Creates new session by providing session details.Example command for Flex shapes : %create_session -l python -c '{""compartmentId"":""Data Flow Run resource compartment OCID"",""displayName"":""SessionApp"",""sparkVersion"":""3.2.1"",""driverShape"":""VM.Standard.E4.Flex"",""executorShape"":""VM.Standard.E4.Flex"",""numExecutors"":1,""driverShapeConfig"":{""ocpus"":1,""memoryInGBs"":16},""executorShapeConfig"":{""ocpus"":1,""memoryInGBs"":16}}' Example command for Spark dynamic allocation : %create_session -l python -c '{""compartmentId"":""Data Flow Run resource compartment OCID"",""displayName"":""SessionApp"",""sparkVersion"":""3.2.1"",""driverShape"":""VM.Standard2.1"",""executorShape"":""VM.Standard2.1"",""numExecutors"":1,""configuration"":{ ""spark.dynamicAllocation.enabled"":""true"", ""spark.dynamicAllocation.shuffleTracking.enabled"":""true"", ""spark.dynamicAllocation.minExecutors"":""1"", ""spark.dynamicAllocation.maxExecutors"":""4"", ""spark.dynamicAllocation.executorIdleTimeout"":""60"", ""spark.dynamicAllocation.schedulerBacklogTimeout"":""60"", ""spark.dataflow.dynamicAllocation.quotaPolicy"":""min"" }}'"
activate_session,"%activate_session -l python -c '{""compartmentId"":""Data Flow Run resource compartment OCID"",""displayName"":""SessionApp"",""applicationId"":""Existing sessionId to activate.""}'",Activate session by providing existing sessionId.
use_session,%use_session -s {sessionId},To use already existing active session.
status,%status,Outputs current session status.
update_session,"%update_session -i '{""maxDurationInMinutes"": 4896,""idleTimeoutInMinutes"": 4888}'",Updates current active session[not session config] for max duration or idle time out.
stop_session,%stop_session,Stops current active session. One active session should be associated with current notebook to stop.
config,%config,Outputs current session configuration.
configure_session,"%configure_session -i '{""driverShape"": ""VM.Standard2.1"", ""executorShape"": ""VM.Standard2.1"", ""numExecutors"": 1}'","Configures the session creation parameters. The force flag -f is mandatory for immediate effect of the config change, in that case session will be dropped and recreated."
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."


In [15]:
%create_session -l python -c $command

Setting up the Cluster..


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cluster is ready..
Starting Spark application..


Session ID,Kind,State,Current session
ocid1.dataflowapplication.oc1.iad.anuwcljsnif7xwiawhoarfxh4eo66zdbssfmltb6y2qfqgaqnifuscw7okwa,pyspark,IN_PROGRESS,Dataflow Run


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
SparkContext available as 'sc'.


In [23]:
%%spark
data = [[295, "South Bend", "Indiana", "IN", 101190, 112.9],[10, "Redwood Shores", "California", "CA", 94065, 200.9]]
columns = ["rank", "city", "state", "code", "population", "price"]

df1 = spark.createDataFrame(data, schema="rank LONG, city STRING, state STRING, code STRING, population LONG, price DOUBLE")
df1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------+----------+----+----------+-----+
|rank|          city|     state|code|population|price|
+----+--------------+----------+----+----------+-----+
| 295|    South Bend|   Indiana|  IN|    101190|112.9|
|  10|Redwood Shores|California|  CA|     94065|200.9|
+----+--------------+----------+----+----------+-----+

In [18]:
%%spark
schemaName = "silverzone"
sqlContext.sql("Create schema " + schemaName)
sqlContext.sql("Use " + schemaName)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [25]:
%%spark
df1.createOrReplaceTempView("df1_view")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
%%spark -c sql -o rslt_view
select  *
from    df1_view; 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

,rank,city,state,code,population,price
0,295,South Bend,Indiana,IN,101190,112.9
1,10,Redwood Shores,California,CA,94065,200.9


In [27]:
from autovizwidget.widget.utils import display_dataframe
display_dataframe(rslt_view)

Output()

AutoVizWidget()

In [28]:
%stop_session

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Session has been stopped successfully.
